### Text classification using NLP / Core engine of a chatbot

* Aim of the project is to design the industrial safety NLP based Chatbot - To design a ML/DL based chatbot utility which can help the professionals to highlight the safety risk as per the incident description by predicting Accident Level/Potential Accident Level.

###### Download
* nltk.download_gui()

#### Note: The above will open a GUI

* Select the below:
* stopwords from Corpa
* averaged_perceptron_tagger from All corpus
* wordnet (OR) you can download all the nltk components by: nltk.download()

* Please Note: The above will take much time (30-60mins depending on Internet speed)

In [82]:
#Import useful libraries
import nltk
import re
import os
import csv
from nltk.stem.snowball import SnowballStemmer
import random
from nltk.classify import SklearnClassifier
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import numpy as np
import pandas as pd
#Visualization
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
## Get multiple outputs in the same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Ignore all warnings
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

In [3]:
## Display all rows and columns of a dataframe instead of a truncated version
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### Pre-processing

In [4]:
# Sentence from the description column
sentence = "While removing the drill rod of the Jumbo 08 for maintenance,the supervisor proceeds to loosen the support of the intermediate centralizer to facilitate the removal, seeing this the mechanic supports one end on the drill of the equipment to pull with both hands the bar and accelerate the removal from this, at this moment the bar slides from its point of support and tightens the fingers of the mechanic between the drilling bar and the beam of the jumbo."
sentence2 = "During the activation of a sodium sulphide pump,the piping was uncoupled and the sulfide solution was designed in the area to reach the maid. Immediately she made use of the emergency shower and was directed to the ambulatory doctor and later to the hospital. Note: of sulphide solution = 48 grams / liter."

In [5]:
#convert sentence to lower case
'This' == 'this'
print('AbcdEFgH'.lower())
sentence.lower()
sentence2.lower()

False

abcdefgh


'while removing the drill rod of the jumbo 08 for maintenance,the supervisor proceeds to loosen the support of the intermediate centralizer to facilitate the removal, seeing this the mechanic supports one end on the drill of the equipment to pull with both hands the bar and accelerate the removal from this, at this moment the bar slides from its point of support and tightens the fingers of the mechanic between the drilling bar and the beam of the jumbo.'

'during the activation of a sodium sulphide pump,the piping was uncoupled and the sulfide solution was designed in the area to reach the maid. immediately she made use of the emergency shower and was directed to the ambulatory doctor and later to the hospital. note: of sulphide solution = 48 grams / liter.'

#### Tokenize - extract individual words

In [6]:
tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(sentence)
tokens
tokens2 = tokenizer.tokenize(sentence2)
tokens2

['While',
 'removing',
 'the',
 'drill',
 'rod',
 'of',
 'the',
 'Jumbo',
 '08',
 'for',
 'maintenance',
 'the',
 'supervisor',
 'proceeds',
 'to',
 'loosen',
 'the',
 'support',
 'of',
 'the',
 'intermediate',
 'centralizer',
 'to',
 'facilitate',
 'the',
 'removal',
 'seeing',
 'this',
 'the',
 'mechanic',
 'supports',
 'one',
 'end',
 'on',
 'the',
 'drill',
 'of',
 'the',
 'equipment',
 'to',
 'pull',
 'with',
 'both',
 'hands',
 'the',
 'bar',
 'and',
 'accelerate',
 'the',
 'removal',
 'from',
 'this',
 'at',
 'this',
 'moment',
 'the',
 'bar',
 'slides',
 'from',
 'its',
 'point',
 'of',
 'support',
 'and',
 'tightens',
 'the',
 'fingers',
 'of',
 'the',
 'mechanic',
 'between',
 'the',
 'drilling',
 'bar',
 'and',
 'the',
 'beam',
 'of',
 'the',
 'jumbo']

['During',
 'the',
 'activation',
 'of',
 'a',
 'sodium',
 'sulphide',
 'pump',
 'the',
 'piping',
 'was',
 'uncoupled',
 'and',
 'the',
 'sulfide',
 'solution',
 'was',
 'designed',
 'in',
 'the',
 'area',
 'to',
 'reach',
 'the',
 'maid',
 'Immediately',
 'she',
 'made',
 'use',
 'of',
 'the',
 'emergency',
 'shower',
 'and',
 'was',
 'directed',
 'to',
 'the',
 'ambulatory',
 'doctor',
 'and',
 'later',
 'to',
 'the',
 'hospital',
 'Note',
 'of',
 'sulphide',
 'solution',
 '48',
 'grams',
 'liter']

#### Stopwords : Filter words to remove non-useful words

In [7]:
filtered_words = [w for w in tokens if not w in stopwords.words('english')]
filtered_words

['While',
 'removing',
 'drill',
 'rod',
 'Jumbo',
 '08',
 'maintenance',
 'supervisor',
 'proceeds',
 'loosen',
 'support',
 'intermediate',
 'centralizer',
 'facilitate',
 'removal',
 'seeing',
 'mechanic',
 'supports',
 'one',
 'end',
 'drill',
 'equipment',
 'pull',
 'hands',
 'bar',
 'accelerate',
 'removal',
 'moment',
 'bar',
 'slides',
 'point',
 'support',
 'tightens',
 'fingers',
 'mechanic',
 'drilling',
 'bar',
 'beam',
 'jumbo']

In [8]:
filtered_words = [w for w in tokens2 if not w in stopwords.words('english')]
filtered_words

['During',
 'activation',
 'sodium',
 'sulphide',
 'pump',
 'piping',
 'uncoupled',
 'sulfide',
 'solution',
 'designed',
 'area',
 'reach',
 'maid',
 'Immediately',
 'made',
 'use',
 'emergency',
 'shower',
 'directed',
 'ambulatory',
 'doctor',
 'later',
 'hospital',
 'Note',
 'sulphide',
 'solution',
 '48',
 'grams',
 'liter']

In [9]:
# Function preprocess
def preprocess(sentence):
    sentence = sentence.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [w for w in tokens if not w in stopwords.words('english')]
    return filtered_words

In [10]:
preprocessed_sentence = preprocess(sentence)
print(preprocessed_sentence)

['removing', 'drill', 'rod', 'jumbo', '08', 'maintenance', 'supervisor', 'proceeds', 'loosen', 'support', 'intermediate', 'centralizer', 'facilitate', 'removal', 'seeing', 'mechanic', 'supports', 'one', 'end', 'drill', 'equipment', 'pull', 'hands', 'bar', 'accelerate', 'removal', 'moment', 'bar', 'slides', 'point', 'support', 'tightens', 'fingers', 'mechanic', 'drilling', 'bar', 'beam', 'jumbo']


In [11]:
preprocess(sentence2)

['activation',
 'sodium',
 'sulphide',
 'pump',
 'piping',
 'uncoupled',
 'sulfide',
 'solution',
 'designed',
 'area',
 'reach',
 'maid',
 'immediately',
 'made',
 'use',
 'emergency',
 'shower',
 'directed',
 'ambulatory',
 'doctor',
 'later',
 'hospital',
 'note',
 'sulphide',
 'solution',
 '48',
 'grams',
 'liter']

#### Tagging

In [12]:
tags = nltk.pos_tag(preprocessed_sentence)
print(tags)

[('removing', 'VBG'), ('drill', 'NN'), ('rod', 'VB'), ('jumbo', 'JJ'), ('08', 'CD'), ('maintenance', 'NN'), ('supervisor', 'NN'), ('proceeds', 'NNS'), ('loosen', 'VBP'), ('support', 'NN'), ('intermediate', 'JJ'), ('centralizer', 'NN'), ('facilitate', 'NN'), ('removal', 'NN'), ('seeing', 'VBG'), ('mechanic', 'JJ'), ('supports', 'NNS'), ('one', 'CD'), ('end', 'NN'), ('drill', 'NN'), ('equipment', 'NN'), ('pull', 'NN'), ('hands', 'VBZ'), ('bar', 'NN'), ('accelerate', 'NN'), ('removal', 'NN'), ('moment', 'NN'), ('bar', 'NN'), ('slides', 'NNS'), ('point', 'VBP'), ('support', 'NN'), ('tightens', 'NNS'), ('fingers', 'NNS'), ('mechanic', 'JJ'), ('drilling', 'VBG'), ('bar', 'NN'), ('beam', 'NN'), ('jumbo', 'NN')]


In [13]:
tags = nltk.pos_tag(preprocess(sentence2))
print(tags)

[('activation', 'NN'), ('sodium', 'NN'), ('sulphide', 'JJ'), ('pump', 'NN'), ('piping', 'VBG'), ('uncoupled', 'JJ'), ('sulfide', 'JJ'), ('solution', 'NN'), ('designed', 'VBN'), ('area', 'NN'), ('reach', 'NN'), ('maid', 'VBD'), ('immediately', 'RB'), ('made', 'VBN'), ('use', 'NN'), ('emergency', 'NN'), ('shower', 'NN'), ('directed', 'VBD'), ('ambulatory', 'JJ'), ('doctor', 'NN'), ('later', 'RBR'), ('hospital', 'JJ'), ('note', 'NN'), ('sulphide', 'JJ'), ('solution', 'NN'), ('48', 'CD'), ('grams', 'NNS'), ('liter', 'RBR')]


##### Extracting only Nouns and Verb nouns

In [14]:
# Function for extracting tags
def extract_tagged(sentences):
    features = []
    for tagged_word in sentences:
        word, tag = tagged_word
        if tag=='NN' or tag == 'VBN' or tag == 'NNS' or tag == 'VBP' or tag == 'RB' or tag == 'VBZ' or tag == 'VBG' or tag =='PRP' or tag == 'JJ':
            features.append(word)
    return features

In [15]:
extract_tagged(tags)

['activation',
 'sodium',
 'sulphide',
 'pump',
 'piping',
 'uncoupled',
 'sulfide',
 'solution',
 'designed',
 'area',
 'reach',
 'immediately',
 'made',
 'use',
 'emergency',
 'shower',
 'ambulatory',
 'doctor',
 'hospital',
 'note',
 'sulphide',
 'solution',
 'grams']

##### stemming/Lemmatize words

In [16]:
# call stemming and lemmatization
lmtzr = WordNetLemmatizer()
stemmer = SnowballStemmer("english")

In [17]:
def extract_feature(text):
    words = preprocess(text)
#     print('words: ',words)
    tags = nltk.pos_tag(words)
#     print('tags: ',tags)
    extracted_features = extract_tagged(tags)
#     print('Extracted features: ',extracted_features)
    stemmed_words = [stemmer.stem(x) for x in extracted_features]
#     print(stemmed_words)

    result = [lmtzr.lemmatize(x) for x in stemmed_words]
   
    return result

In [18]:
sentence

'While removing the drill rod of the Jumbo 08 for maintenance,the supervisor proceeds to loosen the support of the intermediate centralizer to facilitate the removal, seeing this the mechanic supports one end on the drill of the equipment to pull with both hands the bar and accelerate the removal from this, at this moment the bar slides from its point of support and tightens the fingers of the mechanic between the drilling bar and the beam of the jumbo.'

In [19]:
#words after extraction
words = extract_feature(sentence)
print(words)

['remov', 'drill', 'jumbo', 'mainten', 'supervisor', 'proceed', 'loosen', 'support', 'intermedi', 'central', 'facilit', 'remov', 'see', 'mechan', 'support', 'end', 'drill', 'equip', 'pull', 'hand', 'bar', 'acceler', 'remov', 'moment', 'bar', 'slide', 'point', 'support', 'tighten', 'finger', 'mechan', 'drill', 'bar', 'beam', 'jumbo']


#### Implement BOW
* In simple terms, it’s a collection of words to represent a sentence, disregarding the order in which they appear.

In [20]:
def word_feats(words):
    return dict([(word, True) for word in words])

In [21]:
word_feats(words)

{'remov': True,
 'drill': True,
 'jumbo': True,
 'mainten': True,
 'supervisor': True,
 'proceed': True,
 'loosen': True,
 'support': True,
 'intermedi': True,
 'central': True,
 'facilit': True,
 'see': True,
 'mechan': True,
 'end': True,
 'equip': True,
 'pull': True,
 'hand': True,
 'bar': True,
 'acceler': True,
 'moment': True,
 'slide': True,
 'point': True,
 'tighten': True,
 'finger': True,
 'beam': True}

##### Parsing the whole document

In [22]:
def extract_feature_from_doc(data):
    result = []
    corpus = []
    # The responses of the chat bot
    answers = {}
    for (text,category,answer) in data:

        features = extract_feature(text)

        corpus.append(features)
        result.append((word_feats(features), category))
        answers[category] = answer

    return (result, sum(corpus,[]), answers)

In [23]:
extract_feature_from_doc([['this is the input text from the user','category','answer to give']])

([({'input': True, 'user': True}, 'category')],
 ['input', 'user'],
 {'category': 'answer to give'})

In [24]:
def get_content(filename):
    doc = os.path.join(filename)
    with open(doc, 'r') as content_file:
        lines = csv.reader(content_file,delimiter='|')
        data = [x for x in lines if len(x) == 3]
        return data

In [25]:
# Importing the txtfile
filename = '/Users/Lila_Vudumula/OneDrive - EPAM/Desktop/chatbot.txt'
data = get_content(filename)

##### Note: 
* The intents and entity have been created in a text document for training purpose. we can elborate the conversational flow based on the requirement.

In [26]:
data

[['Hello', 'Greetings', 'Hello. I am bot. I will serve your enquiries.'],
 ['hi hello', 'Greetings', 'Hello. I am bot. I will serve your enquiries.'],
 ['hi ', 'Greetings', 'Hello. I am bot. I will serve your enquiries.'],
 ['hi', 'Greetings', 'Hello. I am bot. I will serve your enquiries.'],
 ['hi', 'Greetings', 'Hello. I am bot. I will serve your enquiries.'],
 ['hey', 'Greetings', 'Hello. I am bot. I will serve your enquiries.'],
 ['hello, hi', 'Greetings', 'Hello. I am bot. I will serve your enquiries.'],
 ['hey', 'Greetings', 'Hello. I am bot. I will serve your enquiries.'],
 ['hey, hi', 'Greetings', 'Hello. I am bot. I will serve your enquiries.'],
 ['hey, hello', 'Greetings', 'Hello. I am bot. I will serve your enquiries.'],
 ['Good morning',
  'Morning',
  'Good Morning. I am bot. I will serve your enquiries.'],
 ['Good afternoon',
  'Afternoon',
  'Good afternoon. I am bot. I will serve your enquiries.'],
 ['Good evening',
  'Evening',
  'Good evening. I am bot. I will serve y

In [27]:
features_data, corpus, answers = extract_feature_from_doc(data)

In [28]:
print(features_data[10])

({'good': True, 'morn': True}, 'Morning')


In [29]:
corpus

['hello',
 'hi',
 'hello',
 'hi',
 'hi',
 'hi',
 'hey',
 'hello',
 'hi',
 'hey',
 'hey',
 'hi',
 'hey',
 'hello',
 'good',
 'morn',
 'good',
 'afternoon',
 'good',
 'even',
 'today',
 'want',
 'help',
 'need',
 'help',
 'help',
 'want',
 'help',
 'want',
 'assist',
 'help',
 'great',
 'talk',
 'great',
 'thank',
 'help',
 'thank',
 'thank',
 'much',
 'thank',
 'thank',
 'much',
 'mani',
 'type',
 'accid',
 'type',
 'accid',
 'type',
 'accid',
 'accid',
 'type',
 'accid',
 'type',
 'mani',
 'accid',
 'level',
 'mani',
 'accid',
 'mani',
 'accident_level',
 'mani',
 'accident_level',
 'accident_level',
 'count']

In [30]:
answers

{'Greetings': 'Hello. I am bot. I will serve your enquiries.',
 'Morning': 'Good Morning. I am bot. I will serve your enquiries.',
 'Afternoon': 'Good afternoon. I am bot. I will serve your enquiries.',
 'Evening': 'Good evening. I am bot. I will serve your enquiries.',
 'Opening': "I'm fine! Thank you. How can I help you?",
 'Help': 'How can I help you?',
 'No-Help': 'Ok sir/madam. No problem. Have a nice day.',
 'Closing': "It's glad to know that I have been helpful. Have a good day!",
 'Accidents-Type': 'Currently I know about two: Accident_level and Potential_accident_level accidents.',
 'Default-Utilized-Accident_level-Accidents': 'we have around 5 Accident_levels.',
 'Utilized-Accident_level-Accidents': ' we have around 5 Accident_levels.'}

#### Train a model using these fetures


In [31]:
## split data into train and test sets
split_ratio = 0.8

In [32]:
def split_dataset(data, split_ratio):
    random.shuffle(data)
    data_length = len(data)
    train_split = int(data_length * split_ratio)
    return (data[:train_split]), (data[train_split:])

In [33]:
training_data, test_data = split_dataset(features_data, split_ratio)

In [34]:
training_data

[({'help': True}, 'No-Help'),
 ({'mani': True, 'accid': True, 'level': True},
  'Default-Utilized-Accident_level-Accidents'),
 ({'accid': True, 'type': True}, 'Accidents-Type'),
 ({'mani': True, 'accident_level': True}, 'Utilized-Accident_level-Accidents'),
 ({'hi': True, 'hello': True}, 'Greetings'),
 ({'hey': True, 'hello': True}, 'Greetings'),
 ({'mani': True, 'accid': True}, 'Default-Utilized-Accident_level-Accidents'),
 ({'today': True}, 'Opening'),
 ({'mani': True, 'accident_level': True}, 'Utilized-Accident_level-Accidents'),
 ({'good': True, 'morn': True}, 'Morning'),
 ({'thank': True}, 'Closing'),
 ({'type': True, 'accid': True}, 'Accidents-Type'),
 ({'type': True, 'accid': True}, 'Accidents-Type'),
 ({'thank': True, 'much': True}, 'Closing'),
 ({'hi': True}, 'Greetings'),
 ({'hey': True, 'hi': True}, 'Greetings'),
 ({'thank': True, 'help': True}, 'Closing'),
 ({'accid': True, 'type': True}, 'Accidents-Type'),
 ({'want': True, 'help': True}, 'Help'),
 ({'great': True, 'talk': 

In [35]:
# save the data
np.save('training_data', training_data)
np.save('test_data', test_data)

#### Model Building

In [36]:
def train_using_decision_tree(training_data, test_data):
    classifier = nltk.classify.DecisionTreeClassifier.train(training_data, entropy_cutoff=0.6, support_cutoff=6)
    classifier_name = type(classifier).__name__
    training_set_accuracy = nltk.classify.accuracy(classifier, training_data)
    print('training set accuracy: ', training_set_accuracy)
    test_set_accuracy = nltk.classify.accuracy(classifier, test_data)
    print('test set accuracy: ', test_set_accuracy)
    return classifier, classifier_name, test_set_accuracy, training_set_accuracy

In [37]:
dtclassifier, classifier_name, test_set_accuracy, training_set_accuracy = train_using_decision_tree(training_data, test_data)

training set accuracy:  0.9310344827586207
test set accuracy:  0.625


#### Naive Bayes

In [38]:
def train_using_naive_bayes(training_data, test_data):
    classifier = nltk.NaiveBayesClassifier.train(training_data)
    classifier_name = type(classifier).__name__
    training_set_accuracy = nltk.classify.accuracy(classifier, training_data)
    test_set_accuracy = nltk.classify.accuracy(classifier, test_data)
    return classifier, classifier_name, test_set_accuracy, training_set_accuracy

In [39]:
classifier, classifier_name, test_set_accuracy, training_set_accuracy = train_using_naive_bayes(training_data, test_data)
print(training_set_accuracy)
print(test_set_accuracy)
print(len(classifier.most_informative_features()))
classifier.show_most_informative_features()

0.9655172413793104
0.75
40
Most Informative Features
                    help = True             Help : Closin =      4.1 : 1.0
                    mani = True           Defaul : Accide =      3.3 : 1.0
                     hey = None           Closin : Greeti =      2.6 : 1.0
                   thank = None           Greeti : Closin =      2.6 : 1.0
                      hi = None           Closin : Greeti =      1.9 : 1.0
                   level = None           Closin : Defaul =      1.9 : 1.0
                    need = None           Closin : Help   =      1.5 : 1.0
                    want = None           Closin : Help   =      1.5 : 1.0
                   great = None           Greeti : Closin =      1.4 : 1.0
                   hello = None           Closin : Greeti =      1.4 : 1.0


In [40]:
classifier.classify(({'mani': True, 'option': True, 'leav': True}))

'Utilized-Accident_level-Accidents'

In [41]:
extract_feature("hello")

['hello']

In [42]:
word_feats(extract_feature("hello"))

{'hello': True}

In [43]:
input_sentence = "How many types of accidents are there?"
classifier.classify(word_feats(extract_feature(input_sentence)))

'Accidents-Type'

In [44]:
def reply(input_sentence):
    category = dtclassifier.classify(word_feats(extract_feature(input_sentence)))
    return answers[category]

##### Chatbot
* Note:- Check the chatbot request and response from below cells

In [50]:
reply('Hi')

'Hello. I am bot. I will serve your enquiries.'

In [51]:
reply('How are you today?')

"I'm fine! Thank you. How can I help you?"

In [52]:
reply('How many types of accidents are there?')

'Currently I know about two: Accident_level and Potential_accident_level accidents.'

In [53]:
reply('How many Accident_level have happened')

' we have around 5 Accident_levels.'

In [54]:
reply('thank you')

"It's glad to know that I have been helpful. Have a good day!"

#### DL
* Using Deep learning modoels for chatbot

In [71]:
# Import Libraries
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

In [72]:
with open('data.json') as file:
    data = json.load(file)

In [73]:
#Create a feature
training_sentences = []
training_labels = []
labels = []
responses = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])        
num_classes = len(labels)

In [74]:
# Label encoding
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

LabelEncoder()

In [75]:
# Tokenization
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"
# Token
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [76]:
# Model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
#earlystopping
callback = EarlyStopping(monitor='val_loss',mode='min', verbose=0,patience=3)
#Compile
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [77]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d_5 ( (None, 16)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_15 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_16 (Dense)             (None, 6)                 102       
Total params: 16,646
Trainable params: 16,646
Non-trainable params: 0
_________________________________________________________________


In [79]:
epochs = 300
training_history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs,callbacks=callback)

Epoch 1/300
1/1 [==============================] - 0s 3ms/step - loss: 0.1064 - accuracy: 1.0000
Epoch 2/300
1/1 [==============================] - 0s 2ms/step - loss: 0.1056 - accuracy: 1.0000
Epoch 3/300
1/1 [==============================] - 0s 4ms/step - loss: 0.1047 - accuracy: 1.0000
Epoch 4/300
1/1 [==============================] - 0s 2ms/step - loss: 0.1039 - accuracy: 1.0000
Epoch 5/300
1/1 [==============================] - 0s 4ms/step - loss: 0.1031 - accuracy: 1.0000
Epoch 6/300
1/1 [==============================] - 0s 2ms/step - loss: 0.1022 - accuracy: 1.0000
Epoch 7/300
1/1 [==============================] - 0s 2ms/step - loss: 0.1014 - accuracy: 1.0000
Epoch 8/300
1/1 [==============================] - 0s 6ms/step - loss: 0.1006 - accuracy: 1.0000
Epoch 9/300
1/1 [==============================] - 0s 3ms/step - loss: 0.0998 - accuracy: 1.0000
Epoch 10/300
1/1 [==============================] - 0s 2ms/step - loss: 0.0990 - accuracy: 1.0000
Epoch 11/300
1/1 [===========

In [91]:
# saving model
model.save("chat_model")
# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


#### Conclusion

* Once the model has been developed using an algorithm that gives an acceptable accuracy, this model can be called using to any chatbot UI framework.
* Also, it is recommended to use opensource conversational AI frameworks such as Rasa, Dialogflow for better results.